In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torchvision.models import resnet50

import h5py
import numpy as np

# 1. Dataset定義（すでにあるPCamDatasetを少し改良）
class PCamDataset(Dataset):
    def __init__(self, x_path, y_path, transform=None):
        super().__init__()
        self.x_h5 = h5py.File(x_path, 'r')
        self.y_h5 = h5py.File(y_path, 'r')
        self.transform = transform
        
    def __len__(self):
        return self.x_h5['x'].shape[0]
    
    def __getitem__(self, idx):
        img = self.x_h5['x'][idx]  # shape: (H, W, C) かもしれません。要確認
        label = int(self.y_h5['y'][idx][()])  # または .item()
        
        # 画像の形状やdtypeを調整
        # もしグレースケール(1ch)なら3chに変換（コピーなど）
        if img.ndim == 2:
            img = np.stack([img]*3, axis=-1)
        elif img.shape[2] == 1:
            img = np.concatenate([img]*3, axis=2)
        
        if self.transform:
            img = self.transform(img)
        return img, label

# 2. Transform設定（ResNetの入力サイズ224x224、正規化）
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNetの平均値
                         std=[0.229, 0.224, 0.225])   # ImageNetの標準偏差
])

# 3. Dataset & DataLoader作成
train_dataset = PCamDataset(
    '/home/gotou/Medical/camelyonpatch_level_2_split_train_x.h5',
    '/home/gotou/Medical/camelyonpatch_level_2_split_train_y.h5',
    transform=transform
)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

val_dataset = PCamDataset(
    '/home/gotou/Medical/valid_x_uncompressed.h5',
    '/home/gotou/Medical/valid_y_uncompressed.h5',
    transform=transform
)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

# 4. モデル準備
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = resnet18(pretrained=True)  # 事前学習済みモデルを使うのがおすすめ
num_classes = 2  # 2クラス分類の場合
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# 5. 損失関数とオプティマイザ
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

n_epochs = 10  

model.train()
for epoch in range(n_epochs):
    running_loss = 0.0
    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * imgs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    
    # 評価
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    model.train()  # 次のエポック用に再度 train モードに戻す

    print(f"Epoch {epoch+1}/{n_epochs} | Loss: {epoch_loss:.4f} | Val Acc: {accuracy:.2f}%")



/tmp/ipykernel_135813/696787044.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.y_h5['y'][idx][()])  # または .item()
/tmp/ipykernel_135813/696787044.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.y_h5['y'][idx][()])  # または .item()
/tmp/ipykernel_135813/696787044.py:25: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  label = int(self.y_h5['y'][idx][()])  # または .item()
/tmp/ipykernel_135813/696787044.py:25: DeprecationWarning: